### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta
from botocore.vendored import requests

### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


## Stock Recommendation based on Risk Factor and Performance ##

Conscientious:  AMZN, GOOGL, MSFT
Open Personality:  FSR, FB,
Extraversion:  AAPL  
Agreeableness FSR, FB,

AAPL has High expected returns with High Risk Factor 
FSR has High expected returns with Medium Risk Factor 
FB has Medium expected returns with Medium Risk Factor
AMZN, GOOGL, MSFT have Medium expected returns with Low Risk Factor
TSLA, TWTT &  NVDIA have Low expected returns with Low Risk Factor
INTC  has  Low expected returns with High Risk Factor



def risk(Personality_type):
    """
    Stock Recommendation based on Personnality Type (Risk Factor) and stock Performance 
    """
    if Personality_type == "Conscientiousness":
        rec = "AMZN, GOOGL, MSFT, TSLA, TWTT &  NVDIA (AMZN, GOOGL, MSFT have Medium expected returns with Low Risk Factor and TSLA, TWTT &  NVDIA have Low expected returns with Low Risk Factor "
    elif Personality_type == "Openness":
        rec = "FB (FB has Medium expected returns with Medium Risk Factor)"
    elif Personality_type == "Agreeableness":
        rec = "FSR, (FSR has High expected returns with Medium Risk Factor) "
    elif Personality_type == "Extraversion":
        rec = "AAPL (AAPL has High expected returns with High Risk Factor) "
    else:
        rec = "INTC (INTC  has  Low expected returns with High Risk Factor)"

    return rec


def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }
## DATA VALIDATION ##

def validate_data(age, investment_amount, intent_request):
    """
    Validates the data provided by the user.
    """

    # Validate that the user's age is under 65 years old
    if age is not None:
        age = parse_int(age)
        if age < 21:
            return build_validation_result(
                False,
                "Age",
                "You should be 21 years or older to use this service, "
                "please provide a different age.",
            )

    # Validate the investment amount, it should be >= 1000
    if investment_amount is not None:
        investment_amount = parse_int(
            investment_amount
        )  # Since parameters are strings it's important to cast values
        if investment_amount < 1000:
            return build_validation_result(
                False,
                "investmentAmount",
                "The minimum investment amount is 1,000 USD to use this service, "
                "please provide a greater amount.",
            )

    # A True results is returned if age or amount are valid
    return build_validation_result(True, None, None)


### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }


def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response


### Intents Handlers ###
def recommend_stock(intent_request):
    """
    Performs dialog management and fulfillment for recommending a stock.
    """

    first_name = get_slots(intent_request)["FirstName"]
    age = get_slots(intent_request)["Age"]
    investment_amount = get_slots(intent_request)["investmentAmount"]
    Personality_type = get_slots(intent_request)["PersonalityType"]
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        
        slots = get_slots(intent_request)
        validation_result = validate_data(age, investment_amount, intent_request)
        
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None
            
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attibutes
        output_session_attributes = intent_request["sessionAttributes"]

        return delegate(output_session_attributes, get_slots(intent_request))

    # Get the initial investment recommendation

    initial_recommendation = risk(Personality_type)

    # Return a message with the initial recommendation based on the personality type.
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """{} thank you for your information;
            based on the Personality Type you selected, my recommendation is to choose  the following stock(s) based on the market volatility and Future Performnce  {}
            """.format(
                first_name, initial_recommendation
            ),
        },
    )


### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "Greeting":
        return recommend_stock(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """
    

    return dispatch(event)